In [1]:
import os
import sys
import math
#import itertools
import datetime
import numpy as np
import pandas as pd

import keras

Using TensorFlow backend.


In [2]:
# Append TSPerf path to sys.path
nb_dir = os.path.split(os.getcwd())[0]
tsperf_dir = os.path.dirname(os.path.dirname(os.path.dirname(nb_dir)))
if tsperf_dir not in sys.path:
    sys.path.append(tsperf_dir)

from common.metrics import MAPE
import retail_sales.OrangeJuice_Pt_3Weeks_Weekly.common.benchmark_settings as bs

In [3]:
# Data paths
DATA_DIR = '../../data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

In [4]:
def df_from_cartesian_product(dict_in):
    """Generate a Pandas dataframe from Cartesian product of lists.
    
    Args: 
        dict_in (Dictionary): Dictionary containing multiple lists
        
    Returns:
        df (Dataframe): Dataframe corresponding to the Caresian product of the lists
    """
    from collections import OrderedDict
    from itertools import product
    od = OrderedDict(sorted(dict_in.items()))
    cart = list(product(*od.values()))
    df = pd.DataFrame(cart, columns=od.keys())
    return df

def gen_sequence(df, seq_len, seq_cols, start_timestep=0, end_timestep=None):
    """Reshape features into an array of dimension (time steps, features).  
    
    Args:
        df (Dataframe): Time series data of a specific (store, brand) combination
        seq_len (Integer): The number of previous time series values to use as input features
        seq_cols (List): A list of names of the feature columns 
        start_timestep (Integer): First time step you can use to create feature sequences
        end_timestep (Integer): Last time step you can use to create feature sequences
        
    Returns:
        A generator object for iterating all the feature sequences
    """
    data_array = df[seq_cols].values
    if end_timestep is None:
        end_timestep = df.shape[0]
    for start, stop in zip(range(start_timestep, end_timestep-seq_len), range(seq_len, end_timestep)):
        yield data_array[start:stop, :]


def gen_sequence_array(df_all, seq_len, seq_cols, start_timestep=0, end_timestep=None):
    """Combine feature sequences for all the combinations of (store, brand) into an 3d array.
    
    Args:
        df (Dataframe): Time series data of all stores and brands
        seq_len (Integer): The number of previous time series values to use as input features
        seq_cols (List): A list of names of the feature columns 
        start_timestep (Integer): First time step you can use to create feature sequences
        end_timestep (Integer): Last time step you can use to create feature sequences
        
    Returns:
        seq_array (Numpy Array): An array of the feature sequences of all stores and brands    
    """
    seq_gen = (list(gen_sequence(df_all[(df_all['store']==cur_store) & (df_all['brand']==cur_brand)], \
                                 seq_len, seq_cols, start_timestep, end_timestep)) \
              for cur_store, cur_brand in zip(df_all['store'].unique(), df_all['brand'].unique()))
    seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
    return seq_array

In [5]:
r = 0
print('---- Round ' + str(r+1) + ' ----')
train_df = pd.read_csv(os.path.join(TRAIN_DIR, 'train_round_'+str(r+1)+'.csv'))
train_df['move'] = train_df['logmove'].apply(lambda x: round(math.exp(x)))
train_df.drop('logmove', axis=1, inplace=True)
print(train_df.head(3))
print('')
# Fill missing values
store_list = train_df['store'].unique()
brand_list = train_df['brand'].unique()
week_list = range(bs.TRAIN_START_WEEK, bs.TEST_END_WEEK_LIST[r]+1)
d = {'store': store_list,
     'brand': brand_list,
     'week': week_list}        
data_grid = df_from_cartesian_product(d)
data_filled = pd.merge(data_grid, train_df, how='left', 
                        on=['store', 'brand', 'week'])
print('Number of missing rows is {}'.format(data_filled[data_filled.isnull().any(axis=1)].shape[0]))
print('')
data_filled = data_filled.groupby(['store', 'brand']). \
                          apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
print(data_filled.head(3))
print('')

---- Round 1 ----
   store  brand  week  constant    price1    price2    price3    price4  \
0      2      1    40         1  0.060469  0.060497  0.042031  0.029531   
1      2      1    46         1  0.060469  0.060312  0.045156  0.046719   
2      2      1    47         1  0.060469  0.060312  0.045156  0.046719   

     price5    price6    price7    price8    price9   price10   price11  deal  \
0  0.049531  0.053021  0.038906  0.041406  0.028906  0.024844  0.038984     1   
1  0.049531  0.047813  0.045781  0.027969  0.042969  0.042031  0.038984     0   
2  0.037344  0.053021  0.045781  0.041406  0.048125  0.032656  0.038984     0   

   feat     profit  move  
0   0.0  37.992326  8256  
1   0.0  30.126667  6144  
2   0.0  30.000000  3840  

Number of missing rows is 6204

   brand  store  week  constant    price1    price2    price3    price4  \
0      1      2    40       1.0  0.060469  0.060497  0.042031  0.029531   
1      1      2    41       1.0  0.060469  0.060497  0.042031  0.

In [6]:
print(data_filled)

       brand  store  week  constant    price1    price2    price3    price4  \
0          1      2    40       1.0  0.060469  0.060497  0.042031  0.029531   
1          1      2    41       1.0  0.060469  0.060497  0.042031  0.029531   
2          1      2    42       1.0  0.060469  0.060497  0.042031  0.029531   
3          1      2    43       1.0  0.060469  0.060497  0.042031  0.029531   
4          1      2    44       1.0  0.060469  0.060497  0.042031  0.029531   
5          1      2    45       1.0  0.060469  0.060497  0.042031  0.029531   
6          1      2    46       1.0  0.060469  0.060312  0.045156  0.046719   
7          1      2    47       1.0  0.060469  0.060312  0.045156  0.046719   
8          1      2    48       1.0  0.060469  0.060312  0.049844  0.037344   
9          1      2    49       1.0  0.060469  0.060312  0.049844  0.037344   
10         1      2    50       1.0  0.060469  0.060312  0.043594  0.031094   
11         1      2    51       1.0  0.060469  0.060

In [11]:
data_sub = data_filled[(data_filled.brand==1) & (data_filled.store==2)]
sequence_length = 8
sequence_cols = ['move', 'price1']

In [15]:
data_sub

,brand,store,week,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,move
0,1,2,40,1.0,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1.0,0.0,37.992326,8256.0
1,1,2,41,1.0,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1.0,0.0,37.992326,8256.0
2,1,2,42,1.0,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1.0,0.0,37.992326,8256.0
3,1,2,43,1.0,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1.0,0.0,37.992326,8256.0
4,1,2,44,1.0,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1.0,0.0,37.992326,8256.0
5,1,2,45,1.0,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1.0,0.0,37.992326,8256.0
6,1,2,46,1.0,0.060469,0.060312,0.045156,0.046719,0.049531,0.047813,0.045781,0.027969,0.042969,0.042031,0.038984,0.0,0.0,30.126667,6144.0
7,1,2,47,1.0,0.060469,0.060312,0.045156,0.046719,0.037344,0.053021,0.045781,0.041406,0.048125,0.032656,0.038984,0.0,0.0,30.000000,3840.0
8,1,2,48,1.0,0.060469,0.060312,0.049844,0.037344,0.049531,0.053021,0.045781,0.041406,0.042344,0.032656,0.038984,0.0,0.0,29.950000,8000.0
9,1,2,49,1.0,0.060469,0.060312,0.049844,0.037344,0.049531,0.053021,0.045781,0.041406,0.042344,0.032656,0.038984,0.0,0.0,29.950000,8000.0


In [14]:
# Test MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data_sub)
data_sub_scaled = scaler.transform(data_sub)
data_sub_scaled

array([[0.        , 0.        , 0.        , ..., 0.        , 0.73084502,
        0.11508554],
       [0.        , 0.        , 0.01020408, ..., 0.        , 0.73084502,
        0.11508554],
       [0.        , 0.        , 0.02040816, ..., 0.        , 0.73084502,
        0.11508554],
       ...,
       [0.        , 0.        , 0.97959184, ..., 0.        , 0.3278259 ,
        0.21617418],
       [0.        , 0.        , 0.98979592, ..., 0.        , 0.3278259 ,
        0.21617418],
       [0.        , 0.        , 1.        , ..., 0.        , 0.3278259 ,
        0.21617418]])

In [17]:
data_sub_scaled[0,]

array([0.        , 0.        , 0.        , 0.        , 1.        ,
       1.        , 0.62921348, 0.25477707, 1.        , 1.        ,
       0.70103093, 0.97435897, 0.41148325, 0.35294118, 1.        ,
       1.        , 0.        , 0.73084502, 0.11508554])

In [12]:
sa = gen_sequence_array(data_sub, sequence_length, sequence_cols)
sa.shape

(91, 8, 2)

In [13]:
sa

array([[[8.2560000e+03, 6.0468748e-02],
        [8.2560000e+03, 6.0468748e-02],
        [8.2560000e+03, 6.0468748e-02],
        ...,
        [8.2560000e+03, 6.0468748e-02],
        [6.1440000e+03, 6.0468748e-02],
        [3.8400000e+03, 6.0468748e-02]],

       [[8.2560000e+03, 6.0468748e-02],
        [8.2560000e+03, 6.0468748e-02],
        [8.2560000e+03, 6.0468748e-02],
        ...,
        [6.1440000e+03, 6.0468748e-02],
        [3.8400000e+03, 6.0468748e-02],
        [8.0000000e+03, 6.0468748e-02]],

       [[8.2560000e+03, 6.0468748e-02],
        [8.2560000e+03, 6.0468748e-02],
        [8.2560000e+03, 6.0468748e-02],
        ...,
        [3.8400000e+03, 6.0468748e-02],
        [8.0000000e+03, 6.0468748e-02],
        [8.0000000e+03, 6.0468748e-02]],

       ...,

       [[5.0560000e+03, 4.9843751e-02],
        [4.3584000e+04, 3.1093750e-02],
        [2.5728000e+04, 2.7968749e-02],
        ...,
        [1.5168000e+04, 4.3891560e-02],
        [2.8096000e+04, 3.9062500e-02],
        [

In [10]:
# Model definition
n_steps = 40
n_filters = 32
from keras.models import Sequential, Model
from keras.layers import * #Conv1D, Dense
#sales_in = Input(shape=(n_steps, 1))
#c1 = Conv1D(n_filters, 2, dilation_rate=1, padding='causal', activation='relu')(sales_in)
#c1 = Flatten()(c1)
#output = Dense(2, activation='relu')(c1)
model = Sequential([
    Conv1D(n_filters, 2, dilation_rate=2, padding='causal', activation='relu', input_shape=(n_steps,1)),
    Flatten(),
    Dense(2, activation='relu')
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 40, 32)            96        
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 2,658
Trainable params: 2,658
Non-trainable params: 0
_________________________________________________________________
